# 3 Topic Embedding

This notebook implements a pipeline that produce topic embedding for cognitive tasks and constructs. Topic embedding refers to the weights of each topic for a given task/construct corpus. For example, task A could be assigned the following topic embedding: `[1., .5, .1]` which basically shows the probability of three topics in the task A corpus.

## Input

PubMed abstract corpus, either raw or preprocessed.

## Output

- A list of topics
- embeddings for each task/construct, i.e., probabilities of being assigned to topics.

In [14]:
import pandas as pd
import numpy as np

from IPython.display import display

from bertopic import BERTopic

In [7]:
model = BERTopic.load('outputs/models/pubmedprep_bertopic_pct10_v2021092410.model')

In [35]:
# with pd.option_context('display.max_rows', 10000):
#   display(model.get_topic_info())

# model.visualize_topics()

model.visualize_barchart()